In [1]:
from config import *
import spotipy
import json
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
from sklearn.cluster import KMeans

In [9]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))

In [2]:
song_db=pd.read_csv("data/final_dataset_with_prediction.csv")

In [3]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as file: 
            return pickle.load(file) 
    except FileNotFoundError: 
        print("File not found!") 

In [4]:
scalar = load("scalers/standard.pickle")

In [5]:
model = load("models/kmeans_7.pickle")

In [6]:
def search_song(title, artist, ask_for_options=True):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))
    print("Searching for song {} of artist {}".format(title,artist))
    results = sp.search(q="track:"+title+" artist:"+artist,limit=10)
    if ask_for_options == True:
        if len(results['tracks']['items']) > 1:
            count = 0
            for item in results['tracks']['items']:
                print(count,"Song: '{}', artist: '{}', album: '{}', duration: {} minutes".format(item['name'], item['artists'][0]['name'],item['album']['name'], round((item['duration_ms']/60000),2)))
                count = count + 1
            song_index = int((input("Desirable song (row number): ")))
            return results['tracks']['items'][song_index]['uri'] 

        elif len(results['tracks']['items']) == 1:
            return results['tracks']['items'][0]['uri'] 
        else:
            return ''
    
    if ask_for_options == False:
        if len(results['tracks']['items']) >= 1:
            return results['tracks']['items'][0]['uri']
        else:
            return ''

In [7]:
def is_hot_song(uri):
    return (uri in song_db['uri'].unique()) and ((song_db[song_db['uri'] == uri]['type'] == 'hot')[0])

In [8]:
def search_url(title, artist):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))
    results = sp.search(q="track:"+title+" artist:"+artist,limit=10)
    song_name=results['tracks']['items'][0]['name']
    song_artist=results['tracks']['items'][0]['artists'][0]['name']
    song_url=results['tracks']['items'][0]['external_urls']['spotify']
    return song_name, song_artist, song_url

In [10]:
def get_song():
    answer = True
    while answer == True:
        artist = str(input("Please, enter the name of the artist: "))
        song = str(input("Please, enter the name of the track: "))
        song_id = search_song(song,artist)
        if song_id=='':
            print("Spotify doesn't recognize that song")
            get_song()
        audio_features = pd.DataFrame()
        results = sp.audio_features(song_id)
        audio_features = pd.concat([audio_features, pd.DataFrame(results)])
        y=audio_features[['danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'time_signature']]
        y_scaled=scalar.transform(y)
        pred_cluster=model.predict(y_scaled)
        if is_hot_song(song_id):
            list_of_hot_songs = song_db[song_db['type'] == 'hot']
            hot_same_cluster_songs = list_of_hot_songs[list_of_hot_songs['kmeans_7'] == pred_cluster[0]]
            sample_song = hot_same_cluster_songs.sample()
            sample_artist = sample_song['artist_name']
            sample_track = sample_song['track_name']
            artist_sample = search_url(sample_track, sample_artist)[1]
            track_sample = search_url(sample_track, sample_artist)[0]
            url=search_url(sample_track, sample_artist)[2]
            print(url)
            print("Song: " + track_sample + " from Artist: "+ artist_sample)
        else:
            list_of_not_songs = song_db[song_db['type'] != 'hot']
            not_same_cluster_songs = list_of_not_songs[list_of_not_songs['kmeans_7'] == pred_cluster[0]]
            sample_song = not_same_cluster_songs.sample()
            sample_artist = sample_song['artist_name']
            sample_track = sample_song['track_name']
            artist_sample = search_url(sample_track, sample_artist)[1]
            track_sample = search_url(sample_track, sample_artist)[0]
            url=search_url(sample_track, sample_artist)[2]
            print(url)
            print("Song: " + track_sample + " from Artist: "+ artist_sample)
        user_answer=str(input("Would like to hear another song (yes or no): "))
        if user_answer.lower()=='yes':
            answer=True
        else:
            answer=False

In [11]:
get_song()

Please, enter the name of the artist: queen
Please, enter the name of the track: dont stop
Searching for song dont stop of artist queen
0 Song: 'Don't Stop Me Now - Remastered 2011', artist: 'Queen', album: 'Jazz (Deluxe Remastered Version)', duration: 3.49 minutes
1 Song: 'Don't Stop Me Now', artist: 'Queen', album: 'Jazz', duration: 3.49 minutes
2 Song: 'Don't Stop Me Now - …revisited', artist: 'Queen', album: 'Bohemian Rhapsody (The Original Soundtrack)', duration: 3.63 minutes
3 Song: 'Don't Stop Me Now - Remastered 2011', artist: 'Queen', album: 'Greatest Hits', duration: 3.49 minutes
4 Song: 'Don't Stop Me Now - With Long-Lost Guitars', artist: 'Queen', album: 'Jazz (Deluxe Remastered Version)', duration: 3.57 minutes
5 Song: 'Don't Stop Me Now - Remastered 2011', artist: 'Queen', album: 'The Platinum Collection (Greatest Hits I II & III - 2011 Remaster)', duration: 3.49 minutes
6 Song: 'Don't Stop Me Now', artist: 'Queen', album: 'Discover More Classic Rock', duration: 3.49 minu

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


https://open.spotify.com/track/3lNSX9MR9pzgqKfype7exU
Song: In the Evening by the Moonlight - Mono; Single Edit; 2017 Remaster from Artist: Nina Simone
Would like to hear another song (yes or no): yes
Please, enter the name of the artist: queen
Please, enter the name of the track: dont stop
Searching for song dont stop of artist queen
0 Song: 'Don't Stop Me Now - Remastered 2011', artist: 'Queen', album: 'Jazz (Deluxe Remastered Version)', duration: 3.49 minutes
1 Song: 'Don't Stop Me Now', artist: 'Queen', album: 'Jazz', duration: 3.49 minutes
2 Song: 'Don't Stop Me Now - …revisited', artist: 'Queen', album: 'Bohemian Rhapsody (The Original Soundtrack)', duration: 3.63 minutes
3 Song: 'Don't Stop Me Now - Remastered 2011', artist: 'Queen', album: 'Greatest Hits', duration: 3.49 minutes
4 Song: 'Don't Stop Me Now - With Long-Lost Guitars', artist: 'Queen', album: 'Jazz (Deluxe Remastered Version)', duration: 3.57 minutes
5 Song: 'Don't Stop Me Now - Remastered 2011', artist: 'Queen', a

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


https://open.spotify.com/track/09BZvRoyzf58jpjBStkEzg
Song: ¿Qué Será? / Ritmo Que Pesa / Suelta los Amarres / Revolución en Estéreo / Lo de Más - En Vivo from Artist: Cultura Profética
Would like to hear another song (yes or no): no
